<a href="https://colab.research.google.com/github/TrvKhanh/Vietnamese_sentiment/blob/main/Vietnamese_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets -q
!pip install underthesea -q

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
from underthesea import text_normalize

**PROCESSING DATA**

In [28]:
ds = load_dataset("sepidmnorozy/Vietnamese_sentiment")

def normalize_text(text):
    text = text.lower()
    pattern = r'[^\w\s]'
    text_clear = re.sub(pattern, "", text)
    return text_clear

def correct_text(text):
    text = text_normalize(text)
    return text

def preprocess_text_dataset(dataset):
    dataset = dataset.map(lambda x: {"text": normalize_text(x["text"])})
    dataset = dataset.map(lambda x: {"text": correct_text(x["text"])})
    return dataset

ds = preprocess_text_dataset(ds)

train_dataset = ds['train']
val_dataset = ds['validation']
test_dataset = ds['test']

from transformers import pipeline

llm = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf")

def remove_stopwords_llm(text):
    prompt = f"""
    Hãy giúp tôi loại bỏ những từ không quan trọng (stopwords) khỏi câu sau:
    "{text}"
    Chỉ trả về câu đã được làm sạch, không thêm bất kỳ giải thích nào.
    """
    response = llm(prompt, max_length=200, do_sample=False)
    return response[0]["generated_text"]

# Ví dụ sử dụng
filtered_text = remove_stopwords_llm(text)
print(filtered_text)


Map:   0%|          | 0/2384 [00:00<?, ? examples/s]

Map:   0%|          | 0/331 [00:00<?, ? examples/s]

Map:   0%|          | 0/685 [00:00<?, ? examples/s]

Map:   0%|          | 0/2384 [00:00<?, ? examples/s]

Map:   0%|          | 0/331 [00:00<?, ? examples/s]

Map:   0%|          | 0/685 [00:00<?, ? examples/s]

In [36]:
print(text_normalize("lan dau dung anroid dung lg g2 2 nam nay qua ngon ko 1 loi gi van thich nhat lg"))

lan dau dung anroid dung lg g2 2 nam nay qua ngon ko 1 loi gi van thich nhat lg


In [31]:
for i in range(len(train_dataset)):
    print(train_dataset[i])

{'label': 1, 'text': 'android nhập liệu bằng lời nói rất chuẩn nhanh nhiều ngôn ngữ cùng lúc'}
{'label': 0, 'text': 'sau một thời gian sử dụng aw mình thấy một số điểm chưa hài lòng không đi bơi được ngày nào mình cũng đi tập gym song đi bơi luôn nhưng phải tháo ra rất bực mình do để trong tủ đồ công cộng sợ bị móc chộm quá phụ thuộc vào iphone tính năng nghe gọi nhắn tin hầu như không có tác dụng khi mà iphone kè kè bên mìnhtrừ lúc vào toilet đi ị lúc đó có điện thoại nghe gọi là ổn nhất ở vn ko sài được apple pay tính năng này có cũng như ko mặt aw luôn tối thui như cái đồng hồ rụng kim ps nhình chung dùng cho biết thôi sau một thời gian 34 tháng sẽ thấy nhàm chán và ko sử dụng được nhiều tối về lại phải cắm thêm sạc quên là hôm sau khỏi dùng'}
{'label': 0, 'text': 'phét tôi dùng con 3 gs nguyên bản vào fb lag lòi cả con mắt ra máy chả dám cài cái gì cả chỉ để nghe gọi'}
{'label': 0, 'text': 'bb này ra giá 335 tr có lẽ mới cảm thấy shock chứ giá này thì shoot luôn'}
{'label': 0, 'tex

In [ ]:
model_base = "vinai/phobert-base"

tokenizer = AutoTokenizer.from_pretrained(model_base)
model = AutoModelForSequenceClassification.from_pretrained(model_base, num_labels=2)